<a href="https://colab.research.google.com/github/hamsterjoa/Kbank_playstore_review_analysis/blob/master/Kbank_App_review_preprocessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os, sys  
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install git+https://github.com/haven-jeon/PyKoSpacing.git
!pip install customized_konlpy

In [2]:
import io
import pandas as pd
import numpy as np

import re
import nltk
import pickle
from konlpy.tag import Okt; t=Okt()
from tqdm import tqdm
from pykospacing import Spacing
from collections import Counter
import urllib.request
from tensorflow import keras
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split

In [3]:
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length{length} bytes'.format(name=fn, length=len(uploaded[fn])))

Saving review_dataset.csv to review_dataset (2).csv
User uploaded file "review_dataset.csv" with length553990 bytes


In [27]:
df = pd.read_csv(io.StringIO(uploaded['review_dataset.csv'].decode('utf-8')))
df.drop(['Unnamed: 0','id','date','dateYear','dateMonth','dateDay','userName'], axis = 1, inplace=True)
# strict한 기준으로 판단하기 위해 3점까지는 label 0, 4점부터는 label 1
df['label'] = np.select([df.rating>3],[1],default=0)
# 데이터 확인
df

,rating,content,label
0,1,타 기관 인증서 불러오기가 안되는데 대체 무슨 일이죠?? 범용공동인증서를 등록하려고...,0
1,2,다른건 다 모르겠고 어플 켰을 때 메인 화면이 뜨기까지 시간 좀 더 단축시켜주세요....,0
2,1,타기관인증서를 갱신하면서 인증서가져오기를 수행했는데요. QR로 가져오기는 아예 안돼...,0
3,3,갤럭시쓰는데 알림진동이 안 울립니다.(에티켓모드 아님) 전에 리뷰보니까 다른분들도 ...,0
4,5,뭣같은 앱아이콘을 바꾼건 정말 좋네요. 거기다 묘하게 앱 자체의 속도도 빨라진 느낌...,1
...,...,...,...
2035,1,갤럭시s9+사용 고객입니다.. 케이뱅크 앱이 실행이 안되고 로고만 뜨네요.. 3일내...,0
2036,1,저는 당신들이 너무 싫습니다. 잘쓰고 있던거를 못쓰게 하고 돈도 못보내고 정말 당신...,0
2037,1,접속을 못하네 인출좀 할려고 하니 상담원은 계속 통화중... 지친다,0
2038,1,인증번호자체가 안오네요 삼성갤럭시 S9 최신OS 시간은 오후 8시부터 계속~ 쭉이요...,0


#### 데이터는 분석에 최신 경향을 반영하기 위해 2021년 4월 3일 데이터부터 받아왔습니다. (더 많은 데이터를 받아오려면 Kbank_App_review.ipynb에서 pause_time의 최소, 최댓값을 더 늘리시면 됩니다.)

## **불용어 제거**

In [28]:
def clean_str(text):
  pattern = '([ㄱ-ㅎㅏ-ㅣ]+)'  # 한글 자음, 모음 제거
  text = re.sub(pattern=pattern, repl='', string=text)
  pattern = '<[^>]*>'         # HTML 태그 제거
  text = re.sub(pattern=pattern, repl='', string=text)
  pattern = '[^\w\s]'         # 특수기호제거
  text = re.sub(pattern=pattern, repl='', string=text)
  return text

  def extract_word(text):
    hangul = re.compile('[^가-힣]') 
    result = hangul.sub(' ', text) 
    return result

In [29]:
def extract_word(text):
    hangul = re.compile('[^가-힣]') 
    result = hangul.sub(' ', text) 
    return result

In [30]:
print("Before Extraction : ",df['content'][135])
print("After Extraction : ", extract_word(df['content'][135]))
print("Before Extraction : ",df['content'][206])
print("After Extraction : ", extract_word(df['content'][206]))

Before Extraction :  알림시 진동안울림 !!!, 실행시 파란배경로고 나오기전 자주색+보라색 같은 배경이 먼저나옴 엄청 거슬림, 앱 전체가 너무 무거움 (기기 최신기종임)
After Extraction :  알림시 진동안울림      실행시 파란배경로고 나오기전 자주색 보라색 같은 배경이 먼저나옴 엄청 거슬림  앱 전체가 너무 무거움  기기 최신기종임 
Before Extraction :  업데이트 하고 나니 귀신같이 푸쉬알림 잘울림 ㅋㅋㅋㅋㅋㅋ개뿔 ㅋㅋㅋ 다시 고요해짐 ㅋㅋㅋ
After Extraction :  업데이트 하고 나니 귀신같이 푸쉬알림 잘울림       개뿔     다시 고요해짐    


## **띄어쓰기 고치기**

In [31]:
def no_space(text):
  text1 = re.sub('&nbsp;|&nbsp;|\n|\t|\r', '', text)
  text2 = re.sub('\n\n','', text1)
  return text2

In [32]:
spacing = Spacing()

print("Before Fixing : ",df['content'][123])
print("After Fixing : ", spacing(df['content'][123]))
print("Before Fixing : ",df['content'][258])
print("After Fixing : ", spacing(df['content'][258]))

Before Fixing :  앱딱열면첫페이지가갑자기보이지않아요 괜찮았는데어느순간안되서삭제후다시깔았는데도 첫페이지는하얗게나와요. ㅠ계좌내역등을조회하려면마이페이지쪽으로가야만보여요ㅠㅠ
After Fixing :  앱 딱 열면 첫 페이지가 갑자기 보이지 않아 요 괜찮았는데 어느 순간 안 되서 삭제 후 다시 깔았는데도 첫 페이지는 하얗게 나와요. ㅠ 계좌 내역 등을 조회하려면 마이페이지 쪽으로 가야만 보여 요ㅠㅠ
Before Fixing :  케이뱅크켜면 얼마전부터 네트워크상태가 안좋다면서 다시깔아야해요 그리고는 초기로그인 딱한번만 제대로되요 데이터로도 와이파이로도 장소도바꿔봤지만 계속그러네요 업데이트때문인지 불편해요
After Fixing :  케이뱅크 켜면 얼마 전부터 네트워크 상태가 안 좋다면서 다시 깔아야 해요 그리고는 초기로그인 딱 한 번만 제대로 되요 데이터로도 와이파이로도 장소도 바꿔봤지만 계속 그러네요 업데이트 때문인지 불편해요


In [33]:
# 전체 데이터에 적용
df['content'] = df['content'].apply(lambda x:clean_str(x))
df['content'] = df['content'].apply(lambda x:extract_word(x))
df['content'] = df['content'].apply(lambda x:no_space(x))

## **학습 데이터 구분**

In [34]:
train_data,test_data = train_test_split(df,test_size=.25,random_state=42)

In [35]:
# train용 리뷰 처리
train_number_list=[]
for num in range(len(train_data)):
    train_number_list.append(num)
    

train_data['index']=train_number_list
train_data=train_data.set_index(['index'])

train_data['content'].nunique(),
train_data['rating'].nunique(),
train_data['label'].nunique()

train_data.drop_duplicates(subset=['content'],inplace=True)

In [36]:
print("총 샘플의 개수: ",len(train_data))

총 샘플의 개수:  1530


In [37]:
# test용 리뷰 처리
test_number_list = []

for num in range(len(test_data)):
  test_number_list.append(num)

test_data['index']=test_number_list
test_data=test_data.set_index(['index'])

test_data['content'].nunique(),
test_data['rating'].nunique()
test_data['label'].nunique()

test_data.drop_duplicates(subset=['content'],inplace=True)

In [38]:
print("총 샘플의 개수: ",len(test_data))

총 샘플의 개수:  510


In [39]:
# 전처리 내용 저장
test_data.to_pickle("/content/drive/My Drive/Colab Notebooks/test_data.pkl")
train_data.to_pickle("/content/drive/My Drive/Colab Notebooks/train_data.pkl")

In [40]:
# 파일 다시 불러오기
test_data = pd.read_pickle("/content/drive/My Drive/Colab Notebooks/test_data.pkl")
train_data = pd.read_pickle("/content/drive/My Drive/Colab Notebooks/train_data.pkl")

In [41]:
train_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1530 entries, 0 to 1529
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   rating   1530 non-null   int64 
 1   content  1530 non-null   object
 2   label    1530 non-null   int64 
dtypes: int64(2), object(1)
memory usage: 47.8+ KB


## **Feature Selection**

In [42]:
X = train_data.content # training 데이터에서 피쳐 추출
y = train_data.rating # training 데이터에서 타겟 추출

In [43]:
X.head()

index
0    아으   업데이트 했더니 바이오인증 바뀌었다며 본인 인증 다시해야함 내 명의 폰도 ...
1      이거 서버문제지 아이폰인데 그냥튕기는데 어디서 설정어쩌구저쩌구야 개소리말고 복구하세요
2    원래 페이지 이동시나 로딩시 케이뱅크의  로고가 화면 중앙에서 로딩중이라는걸 알려줬...
3    아니 잘쓰다가 앱이 실행 화면에서 멈춰서 넘어갈 생각을 안합니다 여러번 삭제하고 다...
4    업비트 이벤트 때문에 가입하려고 보니   일부터 된다고 해서   일에 가입하니 다 ...
Name: content, dtype: object

In [44]:
y.head()

index
0    1
1    1
2    2
3    1
4    1
Name: rating, dtype: int64

## **토큰화**

In [45]:
tokenized = [] # 데이터프레임의 한 컬럼으로 추가할 리스트
def token(dataframe):
    for sentence in dataframe['content']: # 전처리된 리뷰들을 하나씩 꺼내옵니다
        tokens = t.morphs(sentence)
        tokenize = " ".join(tokens) # tokens라는 리스트 안의 형태소들을 띄어쓰기로 분리된 하나의 문자열로 join시켜줍니다.
        tokenized.append(tokenize) # 형태소 단위로 띄어쓰기된 문자열을 최종 리스트에 추가해줍니다
        dataframe["tokenized_stem"] = pd.DataFrame(tokenized) # 리스트를 데이터프레임으로 변환해 tokenized_stem라는 컬럼명으로 추가해줍니다.
    
token(train_data)
token(test_data)

In [46]:
train_data.head()

,rating,content,label,tokenized_stem
index,,,,
0,1,아으 업데이트 했더니 바이오인증 바뀌었다며 본인 인증 다시해야함 내 명의 폰도 ...,0,아으 업데이트 했더니 바이오인증 바뀌었다며 본인 인증 다시 해야 함 내 명의 폰 도...
1,1,이거 서버문제지 아이폰인데 그냥튕기는데 어디서 설정어쩌구저쩌구야 개소리말고 복구하세요,0,이 거 서버 문제 지 아이폰 인데 그냥 튕 기는데 어디서 설정 어 쩌구저쩌 구야 개...
2,2,원래 페이지 이동시나 로딩시 케이뱅크의 로고가 화면 중앙에서 로딩중이라는걸 알려줬...,0,원래 페이지 이 동시 나 로딩 시 케이 뱅크 의 로고 가 화면 중앙 에서 로딩 중 ...
3,1,아니 잘쓰다가 앱이 실행 화면에서 멈춰서 넘어갈 생각을 안합니다 여러번 삭제하고 다...,0,아니 잘쓰다가 앱 이 실행 화면 에서 멈춰서 넘어갈 생각 을 안 합니다 여러 번 삭...
4,1,업비트 이벤트 때문에 가입하려고 보니 일부터 된다고 해서 일에 가입하니 다 ...,0,업 비트 이벤트 때문 에 가입 하려고 보니 일 부터 된다고 해서 일 에 가입 하니 ...


## **품사 태깅 (POS Tagging)**

In [47]:
def postagging(dataframe):
    main_pos = [] # 데이터프레임의 새 컬럼이 될 리스트
    for sentence in dataframe['tokenized_stem']: # 리뷰들을 하나씩 가져옵니다
        pos = t.pos(sentence) # 형태소 분석을 진행하고 해당 리스트를 pos라는 변수로 받습니다
        main_words = [word_pos[0] for word_pos in pos if word_pos[1] in ('Adjective', #형용사
                                                                         'Noun', #명사
                                                                         'Adjective', #부사
                                                                         'Verb', #동사
                                                                         'Number',
                                                                         'KoreanParticle'
                                                                         )] # 가져오고자 하는 품사에 해당하면 해당 형태소를 main_words 리스트에 추가합니다.
        main_words_str = " ".join(main_words) # main_words 리스트 안의 형태소들을 띄어쓰기로 분리된 하나의 문자열로 join시켜줍니다.
        main_pos.append(main_words_str) # 선택한 형태소들로 이루어진 문자열을 최종 리스트에 추가해줍니다
        dataframe["main_pos"] = pd.DataFrame(main_pos) # 리스트를 데이터프레임으로 변환해 main_pos라는 컬럼명으로 추가해줍니다.

postagging(train_data)
postagging(test_data)

In [48]:
# 케이, 뱅크, 앱, 어플 외 의미 없다고 판단되는 부사 및 조사 제거
stopwords = ['도', '는', '다', '의', '가', '이', '은', '한', '에', '하', '고', '을', '를', '인', '듯', '과', '와', '네', '들', '듯', '지', '임', '게','요','거','로','으로',
            '것','수','할','하는','제','에서','그','데','번','해도','죠','된','건','바','구','세','안','하고','만','못','재','잘','하고','더','안','때','앱','어플','다른',
             '왜','좀','안되고','뱅크','케이','은행','있습니다','적','하기','입니다','중','했는데','합니다','같아요','인데','저','면','주','해서','부터','뭐','후','하는데',
             '같습니다','그리고']
test_data['tokenized']=test_data['content'].apply(t.morphs)
test_data['tokenized']=test_data['tokenized'].apply(lambda x: [item for item in x if item not in stopwords])
train_data['tokenized']=train_data['content'].apply(t.morphs)
train_data['tokenized']=train_data['tokenized'].apply(lambda x: [item for item in x if item not in stopwords])

In [49]:
train_data.head()

,rating,content,label,tokenized_stem,main_pos,tokenized
index,,,,,,
0,1,아으 업데이트 했더니 바이오인증 바뀌었다며 본인 인증 다시해야함 내 명의 폰도 ...,0,아으 업데이트 했더니 바이오인증 바뀌었다며 본인 인증 다시 해야 함 내 명의 폰 도...,아으 업데이트 했더니 바이오인증 바뀌었다며 본인 인증 다시 해야 함 내 명의 폰 도...,"[아으, 업데이트, 했더니, 바이오인증, 바뀌었다며, 본인, 인증, 다시, 해야, ..."
1,1,이거 서버문제지 아이폰인데 그냥튕기는데 어디서 설정어쩌구저쩌구야 개소리말고 복구하세요,0,이 거 서버 문제 지 아이폰 인데 그냥 튕 기는데 어디서 설정 어 쩌구저쩌 구야 개...,이 거 서버 문제 지 아이폰 그냥 튕 기는데 설정 쩌구저쩌 구야 개 소리 복구 하세요,"[서버, 문제, 아이폰, 그냥, 튕, 기는데, 어디서, 설정, 어, 쩌구저쩌, 구야..."
2,2,원래 페이지 이동시나 로딩시 케이뱅크의 로고가 화면 중앙에서 로딩중이라는걸 알려줬...,0,원래 페이지 이 동시 나 로딩 시 케이 뱅크 의 로고 가 화면 중앙 에서 로딩 중 ...,원래 페이지 이 동시 나 로딩 시 케이 뱅크 의 로고 가 화면 중앙 로딩 중 이라 ...,"[원래, 페이지, 동시, 나, 로딩, 시, 로고, 화면, 중앙, 로딩, 이라, 는걸..."
3,1,아니 잘쓰다가 앱이 실행 화면에서 멈춰서 넘어갈 생각을 안합니다 여러번 삭제하고 다...,0,아니 잘쓰다가 앱 이 실행 화면 에서 멈춰서 넘어갈 생각 을 안 합니다 여러 번 삭...,아니 잘쓰다가 앱 이 실행 화면 멈춰서 넘어갈 생각 안 합니다 여러 번 삭제 하고 ...,"[아니, 잘쓰다가, 실행, 화면, 멈춰서, 넘어갈, 생각, 여러, 삭제, 다시, 깔..."
4,1,업비트 이벤트 때문에 가입하려고 보니 일부터 된다고 해서 일에 가입하니 다 ...,0,업 비트 이벤트 때문 에 가입 하려고 보니 일 부터 된다고 해서 일 에 가입 하니 ...,업 비트 이벤트 때문 가입 하려고 보니 일 부터 된다고 해서 일 가입 하니 끝나있고...,"[업, 비트, 이벤트, 때문, 가입, 하려고, 보니, 일, 된다고, 일, 가입, 하..."


### 다양한 분석을 위해 총 3개의 전처리된 컬럼을 생성하였습니다.

In [50]:
# 내용 저장
test_data.to_pickle("/content/drive/My Drive/Colab Notebooks/test_data_for_modeling.pkl")
train_data.to_pickle("/content/drive/My Drive/Colab Notebooks/train_data_for_modeling.pkl")